In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
path = '/Users/muzaffar/32_items/photas_32'

In [ ]:
def scaledata(datain, minval, maxval):
    dataout = datain - np.min(datain)
    dataout = (dataout / np.ptp(dataout)) * (maxval - minval)
    dataout = dataout + minval
    return dataout

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
file = path + 'data_batch_1'
data_batch_1 = unpickle(file)

In [ ]:
meta_file = path + 'batches.meta'
meta_data = unpickle(meta_file)

In [ ]:
print(data_batch_1.keys())

In [ ]:
images = data_batch_1[b'data']
images = images.reshape(len(images),3,32,32).transpose(0,2,3,1)

In [ ]:
labels = np.array(data_batch_1[b'labels'])
label_names = meta_data[b'label_names']

In [ ]:
for i in range(len(label_names)):
    label_names[i] = label_names[i].decode()

In [ ]:
count_of_class = len(label_names)

In [ ]:
sum(labels == 5)

In [ ]:
def pretty_axis(plt, ax):
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_axis_off()
    plt.tick_params(
        left=False,
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False
    ) # labels along the bottom edge are off

In [ ]:
gg = '123456'

In [ ]:
gg[:5]

In [ ]:
def short_label(lbl, n):
    result = lbl
    if len(lbl) > n:
        result = lbl[:n]+'.'
    return result

In [ ]:
fig = plt.figure(figsize=(count_of_class,2))
gs = gridspec.GridSpec(4,count_of_class, figure=fig)

for i in range(count_of_class):
    idx = np.random.choice(np.where(labels == i)[0],1)[0]
    image = images[idx]
    ax0 = fig.add_subplot(gs[2, i])
    ax0.imshow(image)
    pretty_axis(plt,ax0)
    
    ax1 = fig.add_subplot(gs[0, i])
    ax1.text(0.5, 0.5, labels[idx], fontsize=12,ha='center', va='center')
    pretty_axis(plt,ax1)
    
    name = short_label(label_names[labels[idx]],6)
    ax2 = fig.add_subplot(gs[1, i])
    ax2.text(0.5, 0.5, name, fontsize=12,ha='center', va='center')
    pretty_axis(plt,ax2)
    
    ax3 = fig.add_subplot(gs[3, i])
    ax3.text(0.5, 0.5, sum(labels == i), fontsize=12,ha='center', va='center')
    pretty_axis(plt,ax3)

In [ ]:
min_images_in_class = 10000
for i in range(count_of_class):
    size_of_class = sum(labels == i)
    if (min_images_in_class > size_of_class):
        min_images_in_class = size_of_class

In [ ]:
min_images_in_class

In [ ]:
# https://stackoverflow.com/questions/12201577/how-can-i-convert-an-rgb-image-into-grayscale-in-python

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

test_image_index = 1
test_image = images[test_image_index]
fig = plt.figure(figsize=(2, 2))
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_title('colored\n' + label_names[labels[test_image_index]])
plt.imshow(test_image)

ax1.set_yticklabels([])
ax1.set_xticklabels([])

gray_test_image = rgb2gray(test_image)
ax2 = fig.add_subplot(1, 2, 2)
plt.imshow(gray_test_image, cmap='gray')
ax2.set_title('gray\n' + label_names[labels[test_image_index]])
ax2.set_yticklabels([])
ax2.set_xticklabels([])

In [ ]:
D = np.empty((count_of_class, min_images_in_class), dtype=object)

for i in range(count_of_class):
    idxs = np.where(labels == i)[0]
    for j, image_index in enumerate(idxs):
        if (j < min_images_in_class):
            D[i, j] = scaledata(rgb2gray(images[image_index]),0,1)
        else:
            break;

In [ ]:
wdth = D[0, 0].shape[0]
hdth = D[0, 0].shape[1]

n = wdth * hdth
N = count_of_class * min_images_in_class

X = np.zeros((n, N))
k = 0
for i in range(count_of_class):
    for j in range(min_images_in_class):
        X[:, k] = np.reshape(D[i, j], (wdth*hdth, 1)).T
        k += 1

ones_vector = np.ones((X.shape[1], 1))
Xt = X - X.dot(ones_vector).dot(ones_vector.T)/N

In [ ]:
bottle_nack = 0.3
p = int(bottle_nack * n)

In [ ]:
p

In [ ]:
U, S, V = np.linalg.svd(Xt, full_matrices=False)

UU = U[:, :p]
#T = np.random.rand(p, p) * 0.01
T = np.eye(p,p) * 0.001

In [ ]:
# https://ru.wikipedia.org/wiki/Функция_Хевисайда
# https://ru.wikipedia.org/wiki/Функция_активации

def activation(u):
    coeff_a = 0
    dx = 0.0000001
    coeff_k1 = 10/dx #0.5
    retval = 1 / (1 + np.exp(-2.0 * coeff_k1 * (u + coeff_a)))
    return retval

coeff_a = 0;
dx = 0.0000001
coeff_k1 = 10/dx

alpha0 = 1 / (np.exp(-2*coeff_a*coeff_k1) + 1);
alpha1 = 2 * coeff_k1 * np.exp(2*coeff_a*coeff_k1) / ( np.exp(2*coeff_a*coeff_k1) + 1 )**2;

In [ ]:
W1 = (1/alpha1) * T @ UU.T
W2 = UU @ np.linalg.inv(T)

w1 = -(1/alpha1) * T @ UU.T @ (X @ np.ones((N, 1)) / N)
w2 = (np.eye(n) - UU @ UU.T) @ (X @ np.ones((N, 1)) / N) - alpha0 * UU @ np.linalg.inv(T) @ np.ones((p, 1)) - alpha1 * UU @ np.linalg.inv(T) @ w1


In [ ]:
XX = np.empty((hdth, 0))
YY = np.empty((hdth, 0))

fig, ((ax1), (ax2)) = plt.subplots(2, 1,figsize=(10, 2), dpi=180)

for digit in range(10):
    image_num = np.random.randint(min_images_in_class)
    B = np.dot(W1, D[digit, image_num].reshape(n, 1)) + w1
    H = activation(B)
    Y = np.dot(W2, H) + w2
    XX = np.hstack((XX, D[digit, image_num].reshape(wdth, hdth)))
    YY = np.hstack((YY, Y.reshape(wdth, hdth)))
    
ax1.imshow(XX, cmap='gray', aspect='equal')
ax1.axis('off')
ax2.imshow(YY, cmap='gray', aspect='equal')
ax2.axis('off')